## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [15:12,  8.22it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7500/7500 [00:36<00:00, 203.45it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,NHS B'ham & Solihull CCG,"Birmingham, England",NHS Birmingham and Solihull CCG became the lar...,2009-03-03 14:53:03,12032,3132,2493,True,2020-08-16 09:28:04,If you feel unwell before you're meant to go o...,[COVID19],Sprout Social,False
0,CGTN,"Beijing, China",#SeeTheDifference with CGTN as we bring you st...,2013-01-24 03:18:59,13870120,69,126,True,2020-08-16 09:28:02,"Guangzhou, S #China's Guangdong Province, anno...",[China],Twitter Web App,False
0,Migrants' Rights Network,London,A UK charity campaigning for the rights & just...,2010-02-04 17:16:31,31488,3140,4969,False,2020-08-16 09:28:00,"Are you a person of colour, EU citizen or migr...",None,TweetDeck,False
0,XRKettering,"Kettering, England",Official twitter of Extinction Rebellion Kette...,2019-07-21 00:49:22,519,353,1733,False,2020-08-16 09:27:59,Why should it be a death sentence if you live ...,[COVID19],TweetDeck,False
0,Jules Berkhof,,,2010-01-16 08:24:51,56,291,682,False,2020-08-16 09:27:57,@JumboVismaRoad @rogla Bad luck day for cyclin...,None,Twitter for iPhone,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (132601, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 132601 all tweets: 140101


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (140101, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)